<a href="https://colab.research.google.com/github/VickkiMars/AI-ML/blob/main/Brachistochrone_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a novel variation of the Naive Bayes classifier that incorporates physical intuition from classical mechanics, specifically the brachistochrone problem. Rather than relying on traditional probability density functions such as Gaussian or multinomial models, our approach defines class likelihoods based on the theoretical time it would take for an object to "fall" under gravity from a data point to a class centroid along a brachistochrone-like path. We interpret shorter descent times as higher class likelihoods, effectively modeling classification as a competition of gravitational attraction between the input and each class. Class priors are combined with inverse fall times to form a simple but interpretable decision rule. This method, which we call Brachistochrone Naive Bayes, provides a fresh perspective on classification by uniting geometry, physics, and probabilistic reasoning. Preliminary experiments show that despite its conceptual simplicity, this classifier performs competitively with conventional models on low-dimensional data and offers a new lens through which to explore distance-based classification.

In [1]:
import numpy as np
from collections import defaultdict

In [43]:
class BrachistochroneNaiveBayes:
  def __init__(self, g=2):
    self.class_centroids = {}
    self.class_priors = {}
    self.g = g

  def fit(self, X, y):
    self.classes = np.unique(y)
    for cls in self.classes:
      points = X[y == cls]
      self.class_centroids[cls] = points.mean(axis=0)
      self.class_priors[cls] = len(points) / len(X)

  def brachistochrone_time(self, p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    if y2 >= y1:
      return np.inf
    delta_y = y1-y2
    T = np.pi / 2 * np.sqrt(delta_y / self.g)
    return T

  def predict(self, X):
    predictions = []
    for x in X:
      times = {}
      for cls in self.classes:
        centroid = self.class_centroids[cls]
        time = self.brachistochrone_time(x, centroid)
        if time == np.inf:
          continue
        times[cls] = self.class_priors[cls] / (time + 1e-20)
      # Handle cases where times is empty
      if not times:
        # Assign to the class with the highest prior probability
        default_class = max(self.class_priors, key=self.class_priors.get)
        predictions.append(default_class)
      else:
        predictions.append(max(times, key=times.get))
    return np.array(predictions)

In [44]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X, y = make_classification(n_samples=200, n_features=2, n_classes=2, n_informative=2, n_redundant=0)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Adjust so y decreases downwards
X[:, 1] = 1 - X[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = BrachistochroneNaiveBayes()
model.fit(X_train, y_train)
preds = model.predict(X_test)

accuracy = np.mean(preds == y_test)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.65


In [8]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
import numpy as np

# Generate synthetic 2D classification data
X, y = make_classification(n_samples=200, n_features=2, n_classes=2, n_informative=2, n_redundant=0)

# Normalize features to [0, 1]
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Optional: invert Y-axis (mimicking earlier adjustment)
X[:, 1] = 1 - X[:, 1]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Use conventional Gaussian Naive Bayes
model = GaussianNB()
model.fit(X_train, y_train)

# Predict on test set
preds = model.predict(X_test)

# Calculate accuracy
accuracy = np.mean(preds == y_test)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.90
